# Initial setup

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

Import all needed packages

In [ ]:
import os
import cv2
import tensorflow as tf
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import json
from six.moves import urllib
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.utils import shuffle
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

In [ ]:
from keras import applications
preprocess_input = applications.mobilenet_v2.preprocess_input 
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

Run models on GPU 1

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
tf.config.experimental.set_memory_growth(gpus[1], True)

In [ ]:
#! unzip ~/Documents/Dataset/binary/trainBN400.zip -d ~/Documents/Dataset/binary

## Set useful paths

The folder structure is the following: there is a main folder *Dataset* that contains all sub folders where taking picture to pre-process (*data_test* and *data_test2_esterni*) and where putting all pre-processed ones (*testBN* and *test2BN_esterni*).

In [ ]:
path_ds = "Dataset"

## Load trained model

In [ ]:
path_model = os.path.join(path_ds, "my_model_binary200_400.h5") 

In [ ]:
model_tot = load_model(path_model)

We visualize properties of all layers that are part of the *model_tot*

In [ ]:
model_tot.summary()

# Testing part

## Creation of  predictions *output_test* and true labels *Y_test* (in binary classification labels are 1: target class = Person, 0: alien class, no people inside)

In this part we generate output from test images. The two grayscale dataset contain both 1000 pictures with people and 1000 pictures without individuals.

## Pre-process test images

Folders *data_test* and *data_test2_esterni* contain RGB images of the two categories to pre-process e to store respectively in folders *test* and *test2BN_esterni*.

The structure of folders is the following:
<pre>
<b>data_test or data_test2_esterni</b>
|__ <b>Persona</b>
|__ <b>Others</b>
</pre>

<pre>
<b>testBN or test2BN_esterni</b>
|__ <b>Persona</b>
   |__ <b>0</b>
|__ <b>Others</b>
   |__ <b>1</b>
<pre>

Set useful paths

In [ ]:
path_data_test = os.path.join(path_ds, "data_test") 
path_test = os.path.join(path_ds, "testBN")

#path_data_test = os.path.join(path_ds, "data_test2_esterni") 
#path_test = os.path.join(path_ds, "test2BN_esterni")

path_test_Persona = os.path.join(path_test, "Persona")

path_test_Others = os.path.join(path_test, "Others")

Pre-processing:

• each image is centrally cropped along its smaller size. In this way we
can resize it without altering the image aspect ratio and the properties
of objects within;

• each picture is resized to square format of 224×224 with a bilinear interpolation;

• each image is made a grayscale image with size of (224, 224, 1), having a
single channel;

• each grayscale image is brought back on three channels, repeating the single channel three times. This operation is done since the structure of
most of networks presents a three channel configuration.

In [ ]:
!rm -rf `find -type d -name .ipynb_checkpoints`

In [ ]:
target = "Persona"
img_size = 224
for folder in os.listdir(path_data_test):
    path_folder = os.path.join(path_data_test, folder)
    print("\n------------------------------------------------------")
    print("\nFolder ", folder, " with ", len(os.listdir(path_folder)), "images inside")

    if folder == target:
        path_out = os.path.join(path_test_Persona, "0")
        if not os.path.exists(path_out):
            os.makedirs(path_out)
    else: 
        path_out = os.path.join(path_test_Others, "1")
        if not os.path.exists(path_out):
            os.makedirs(path_out)

    i=0   #new images
    j=0   #images already pre-processed
    for file in os.listdir(path_folder):
        if os.path.exists(path_out + "/" + file):
            j+=1
            print("Image " + file + " already pre-processed" )
        else:
            i+=1
            print("Processing ... ", file)
            
            #read the image
            image = cv2.imread(path_folder + "/"+ file)
            #crop image -> square image along its min dimension
            h, w, c = image.shape
            if w>h:
                start = (w-h)//2
                image = image[:, start:start+h]
            else:
                start = (h-w)//2
                image = image[start:start+w,:]
            #resize
            image = cv2.resize(image, (img_size, img_size), interpolation=cv2.INTER_LINEAR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)     #gray image
            image = cv2.merge((image, image, image))            #gray image on 3 channels
            #write the pre-proc image in train folder
            cv2.imwrite(path_out + "/" + file, image)

    print("\nImages that have been previously pre-processed: " + str(j))
    print("\nNewly pre-processed images: " + str(i))
  

Create predictions *y_Persona* for test images = Persona images using *test_datagen0*

In [ ]:
test_datagen0 = ImageDataGenerator(preprocessing_function = preprocess_input)

test_generator0 = test_datagen0.flow_from_directory(path_test_Persona,
                                                  target_size=(224, 224),
                                                  shuffle = False,
                                                  class_mode='categorical',
                                                  batch_size= 100                                         
                                                  ) 

In [ ]:
y_Persona = model_tot.predict(test_generator0, steps = len(os.listdir(os.path.join(path_test_Persona, "0"))) // 100)

In [ ]:
y_Persona.shape

Create predictions *y_Others* for test images = Others images using *test_datagen1*

In [ ]:
test_datagen1 = ImageDataGenerator(preprocessing_function = preprocess_input)

test_generator1 = test_datagen1.flow_from_directory(path_test_Others,
                                                  target_size=(224, 224),
                                                  shuffle = False,
                                                  class_mode='categorical',
                                                  batch_size= 100 
                                                  ) 

In [ ]:
y_Others = model_tot.predict(test_generator1, steps = len(os.listdir(os.path.join(path_test_Others, "1"))) // 100)

In [ ]:
y_Others.shape

Append all predictions in *output_test*

In [ ]:
output_test = np.concatenate([y_Persona, y_Others])

In [ ]:
output_test.shape

Create true labels of test images in *Y_test*

In [ ]:
Y_test=np.concatenate([np.ones(y_Persona.shape[0]), np.zeros(y_Others.shape[0])])   #11111 ... 00000

In [ ]:
Y_test.shape

## Plot ROC curve

The Receiver Operating Characteristic curve plots the True Positive Rate (TPR) versus the False Positive Rate (FPR) for all possible thresholds. It is used to evaluate DOC models.
Best ones have ROC curves very close to the top left corner of
the plot.

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
#from sklearn.metrics import roc_auc_score

In [ ]:
def plot_roc_curve(fpr, tpr, label):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.legend()
    plt.title('ROC curve')
    plt.plot([0, 1], [0, 1], 'k--') # Dashed diagonal
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.grid(True)

In [ ]:
fpr, tpr, thresholds = roc_curve(Y_test, output_test)
AUC = roc_auc_score(Y_test, output_test)
print(AUC)

In [ ]:
plot_roc_curve(fpr, tpr, label='BinaryClassification(AUC = %.2f)'%AUC)
plt.show()

Save fpr and tpr in folder *metrics* to retrieve them for plots

In [ ]:
path_metrics = os.path.join(path_ds, "metrics")

path_fpr = os.path.join(path_metrics, "fpr200b.npy") 
path_tpr = os.path.join(path_metrics, "tpr200b.npy")
np.save(path_fpr, fpr, allow_pickle=True, fix_imports=True)
np.save(path_tpr, tpr, allow_pickle=True, fix_imports=True)

## Optimal threshold and binary output *y_pred*

Scores are transformed in binary output thanks to a threshold delta. Remember that labels in binary classification are 1: person and 0:others. The positive class is the target class here.

The chosen $\delta$ the one that maximizes the quantity (TPR-FPR), producing an high value of TPR, the True Positive Rate, and a low value of FPR, the False Positive Rate.
The first one indicates the ratio of positive instances correctly classified as positive, while the second one is the ratio of negative instance incorrectly classified.
Therefore, maximizing the term (TPR-FPR) allows to reach an high value of intances classified as people that are actually people and a low value of alien objects wrongly classified as people.
These quantities have been already computed by *roc_curve* command.

In [ ]:
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
print("Threshold value is:", optimal_threshold)

In [ ]:
y_pred = np.zeros(output_test.shape[0])
for i in range(output_test.shape[0]):
    if output_test[i] > optimal_threshold:
        y_pred[i] = 1

In [ ]:
y_pred.shape

Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_pred)
print(cm)
#           predicted               0:negative - others   1:positive - persona
           #   0  1                   FP=false positive, actual others but predicted persona
#actual    #0 TN FP                   FN=false negative, actual persona but predicted others
           #1 FN TP                   

F1score

In [ ]:
from sklearn.metrics import f1_score
f1_score(Y_test, y_pred)

Precision

In [ ]:
from sklearn.metrics import precision_score
precision_score(Y_test, y_pred)

Recall

In [ ]:
from sklearn.metrics import recall_score
recall_score(Y_test, y_pred)

Accuracy

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, y_pred)

## Plot DET curve

The Detection Error Tradeoff curve plots the False Positive Rate (FPR) against the False Negative Rate (FNR) for all possible threshold values

In [ ]:
fps=fpr
fns=1-tpr

In [ ]:
from matplotlib import pyplot as plt
def DETCurve(fps,fns):
    """
    Given false positive and false negative rates, produce a DET Curve.
    The false positive rate is assumed to be increasing while the false
    negative rate is assumed to be decreasing.
    """
    axis_min = min(fps[0],fns[-1])
    fig,ax = plt.subplots()
    plt.plot(fps,fns)
    plt.yscale('log')
    plt.xscale('log')
    plt.xlabel('False Positive Rate (%)')
    plt.ylabel('False Negative Rate (%)')
    ticks_to_use = [0.001,0.002,0.005,0.01,0.02,0.05,0.1,0.2,0.5,1,2,5,10,20,50]
    ax.get_xaxis().set_major_formatter(plt.matplotlib.ticker.ScalarFormatter())
    ax.get_yaxis().set_major_formatter(plt.matplotlib.ticker.ScalarFormatter())
    ax.set_xticks(ticks_to_use)
    ax.set_yticks(ticks_to_use)
    plt.axis([0.001,50,0.001,50])
    plt.grid(True)

DETCurve(fps,fns)

## See features of binary classification

From *model_tot* we take *model_features* for feature extraction

In [ ]:
model_features = Model(model_tot.inputs, model_tot.layers[-2].output) #output = <tf.Tensor 'global_average_pooling2d_1/Identity:0' shape=(None, 1280) dtype=float32>

We visualize properties of all layers from the *model_features*

In [ ]:
model_features.summary()

Extract features from Person images using *test_datagen0*

In [ ]:
features_Persona = model_features.predict(test_generator0, steps = len(os.listdir(os.path.join(path_test_Persona, "0"))) // 100)

In [ ]:
features_Persona.shape

Extract features from Others images using *test_datagen1*

In [ ]:
features_Others = model_features.predict(test_generator1, steps = len(os.listdir(os.path.join(path_test_Others, "1"))) // 100)

In [ ]:
features_Others.shape

Append all features in *features_test*

In [ ]:
features_test = np.concatenate([features_Persona, features_Others]) 

In [ ]:
features_test.shape

Create true labels of test images in *Y_test*

In [ ]:
Y_test=np.concatenate([np.ones(features_Persona.shape[0]), np.zeros(features_Others.shape[0])])

Implement t-SNE visualization of 1280 features extracted from each test image.

• red points with labels 0 are the features associated to images containg people;

• green points labeled with 1 are the features extracted from pictures with no people.

In [ ]:
from __future__ import print_function
import time
import numpy as np
import pandas as pd
#from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

feat_cols = [ 'pixel'+str(i) for i in range(features_test.shape[1]) ]
df = pd.DataFrame(features_test,columns=feat_cols)
df['y'] = Y_test

#feat_cols = [ 'pixel'+str(i) for i in range(features_test_tsne.shape[1]) ]
#df = pd.DataFrame(features_test_tsne,columns=feat_cols)
#df['y'] = Y_test_tsne

df['label'] = df['y'].apply(lambda i: str(i))
#features_test, Y_test = None, None
print('Size of the dataframe: {}'.format(df.shape))

# For reproducability of the results
np.random.seed(6)
rndperm = np.random.permutation(df.shape[0])

N = Y_test.shape[0]
df_subset = df.loc[rndperm[:N],:].copy()
data_subset = df_subset[feat_cols].values
#pca = PCA(n_components=3)
#pca_result = pca.fit_transform(data_subset)
#df_subset['pca-one'] = pca_result[:,0]
#df_subset['pca-two'] = pca_result[:,1] 
#df_subset['pca-three'] = pca_result[:,2]
#print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(data_subset)
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

df_subset['tsne-2d-one'] = tsne_results[:,0]
df_subset['tsne-2d-two'] = tsne_results[:,1]
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="y",
    palette=sns.color_palette("hls", 3),
    data=df_subset,
    legend="full",
    alpha=1
)